In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc,silhouette_score 
from sklearn.cluster import KMeans,DBSCAN
from scipy.spatial import distance
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(2)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


2


In [2]:
rootdir = '/data/fjsdata/physionet/MIT-BIH/mitdb/'
rootdir = '/data/fjsdata/physionet/MIT-BIH/mitdb/'
right_len = 180 #right sample length around of peak value of QRS
left_len = 180 #left sample length around of peak value of QRS
#train data
trPVC = [] #[Subject,sig_name,QRS]
for bt in [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230,\
            100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]:
    #44 records for train
    file = os.path.join(rootdir,str(bt))
    try:
        annotation = wfdb.rdann(file, 'atr') 
        qrs_spl = annotation.sample #numpy.ndarray
        qrs_sym = annotation.symbol #list
        record = wfdb.rdrecord(file)
        signal = record.p_signal #numpy.ndarray
        max_len = record.sig_len #length of samples
        lead_name =  record.sig_name #names of lead channels,list
        for i in range(annotation.ann_len):
            if qrs_sym[i] in ['V']:#PVC samples
                pos = qrs_spl[i] #corresponding position of peak value of QRS
                if pos+right_len<=max_len and pos-left_len>=0:
                    max_idx = pos+right_len#np.min([max_len, pos+trunc_len])
                    min_idx = pos-left_len#np.max([0, pos-trunc_len])
                    for j, val in enumerate(lead_name):
                        if val == 'MLII':
                            QRS_value = signal[:,j][min_idx:max_idx]
                            trPVC.append([bt,QRS_value])#[Subject,sig_name,QRS]
    except: pass
trPVC = pd.DataFrame(np.array(trPVC),columns=['sub','qrs'])
trPVC_QRS = pd.DataFrame(trPVC['qrs'].values.tolist()) #QRS extrend
trPVC = trPVC.drop(['qrs'],axis=1) #drop column 2
trPVC = pd.concat([trPVC, trPVC_QRS], axis=1)
print('The shape of MIT-BIH for PVC trainset is: (%d,%d)'%(trPVC.shape[0],trPVC.shape[1]))

#test data
txt_dir = '/data/fjsdata/ECG/PVC/MIT-BIH-label/' #the path of images
sgl_dir = '/data/fjsdata/physionet/MIT-BIH/mitdb/'
tePVC = []#[Subject,QRS,PVC label]
for iname in os.listdir(txt_dir):
    #read label 
    txt_path = os.path.join(txt_dir, iname) 
    label_list = []
    with open(txt_path,'r') as f:
        for line in f:
            label_list.append(line.strip('\n'))
    bt = os.path.splitext(iname)[0]
    sgl_path = os.path.join(sgl_dir,str(bt))
    #read positon of PVC
    annotation = wfdb.rdann(sgl_path, 'atr') 
    qrs_spl = annotation.sample #numpy.ndarray
    qrs_sym = annotation.symbol #list
    pos_list = []
    for i in range(annotation.ann_len):
        if qrs_sym[i] in ['V']:#PVC samples
            pos = qrs_spl[i] #corresponding position of peak value of QRS
            pos_list.append(qrs_spl[i])
    #read signal        
    record = wfdb.rdrecord(sgl_path)
    signal = record.p_signal #numpy.ndarray
    max_len = record.sig_len #length of samples
    lead_name =  record.sig_name #names of lead channels,list
    for i in range(len(label_list)):
        pos = pos_list[i]
        if pos+right_len<=max_len and pos-left_len>=0:
            max_idx = pos+right_len#np.min([max_len, pos+trunc_len])
            min_idx = pos-left_len#np.max([0, pos-trunc_len])
            for j, val in enumerate(lead_name):
                if val == 'MLII':
                    QRS_value = signal[:,j][min_idx:max_idx]
                    tePVC.append([bt,label_list[i],QRS_value])
tePVC = pd.DataFrame(np.array(tePVC),columns=['sub','label','qrs'])
tePVC_QRS = pd.DataFrame(tePVC['qrs'].values.tolist()) #QRS extrend
tePVC = tePVC.drop(['qrs'],axis=1) #drop column 2
tePVC = pd.concat([tePVC, tePVC_QRS], axis=1)
print('The shape of MIT-BIH for PVC testset is: (%d,%d)'%(tePVC.shape[0],tePVC.shape[1]))

The shape of MIT-BIH for PVC trainset is: (6902,361)
The shape of MIT-BIH for PVC testset is: (1996,362)


In [4]:
#Hanmming Distance
def tanh(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

#Testing hamming distance with true label
sub_list = list(set(np.array(tePVC['sub']).tolist()))
y_pred, y_true = [], []
for bt in sub_list:
    sub_df =  tePVC[tePVC['sub']==bt]
    if (sub_df.shape[0]>1): 
        y = list(set(np.array(sub_df['label']).tolist()))
        if len(y)==1: y_true.append(0) #unifocal
        else:  y_true.append(1) #multifocal
        #calculate hanmming distance
        X = sub_df.drop(['sub','label'],axis=1).reset_index(drop=True) 
        X = np.sign(tanh(np.array(X))) 
        k = 1
        for i in range(X.shape[0]):
            for j in range(i+1,X.shape[0]):
                dist = pdist(np.vstack([X[i],X[j]]),'hamming')
                if dist>0.3: k = k+1
        if k==1: y_pred.append(0) #unifocal
        else:  y_pred.append(1) #multifocal
            
print ( 'Accuracy: %.6f'%accuracy_score(y_true, y_pred))
cm = confusion_matrix(y_true, y_pred, labels=[0,1] ) 
print (cm) 
print ('Sensitivity of unifocal pvc: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of multifocal pvc: %.6f'%float(cm[1][1]/np.sum(cm[1])))

Accuracy: 0.857143
[[ 5  2]
 [ 2 19]]
Sensitivity of unifocal pvc: 0.714286
Sensitivity of multifocal pvc: 0.904762


In [13]:
#Unsupervised Hashing Methods: autoencoder
#https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(360, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 360), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
best_net, best_loss = None, float('inf')
batchSize = 100
num_epochs = 200
X = tePVC.drop(['sub','label'],axis=1).reset_index(drop=True) 
X = np.array(X)
for epoch in range(num_epochs):
    losses = []
    num_batches = X.shape[0] // batchSize + 1
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([X.shape[0], (i+1)*batchSize])
        inputs = torch.from_numpy(X[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        # ===================forward=====================
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
        losses.append(loss.item())
    #print('epoch [{}/{}], loss:{:.6f}'.format(epoch + 1, num_epochs, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))    
#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()    

#Testing hamming distance with true label
sub_list = list(set(np.array(tePVC['sub']).tolist()))
y_pred, y_true = [], []
#pdist = nn.PairwiseDistance(2)
for bt in sub_list:
    sub_df =  tePVC[tePVC['sub']==bt]
    if (sub_df.shape[0]>1): 
        y = list(set(np.array(sub_df['label']).tolist()))
        if len(y)==1: y_true.append(0) #unifocal
        else:  y_true.append(1) #multifocal
        #predict
        X = sub_df.drop(['sub','label'],axis=1).reset_index(drop=True) 
        X = np.array(X)
        k = 1
        for i in range(X.shape[0]-1):
            Q_batch = torch.from_numpy(X[i:i+1]).type(torch.FloatTensor).cuda()
            N_batch = torch.from_numpy(X[i+1:i+2]).type(torch.FloatTensor).cuda()
            Q_hash = torch.sign(best_net(Q_batch)).cpu().detach().numpy()
            N_hash = torch.sign(best_net(N_batch)).cpu().detach().numpy()
            #loss_neg = torch.mean(pdist(Q_hash, N_hash)).item()
            dist = pdist(np.vstack([Q_hash,N_hash]),'hamming')
            if dist>0.2: 
                k = k+1
                break
        if k==1: y_pred.append(0) #unifocal
        else:  y_pred.append(1) #multifocal
            
print ( 'Accuracy: %.6f'%accuracy_score(y_true, y_pred))
cm = confusion_matrix(y_true, y_pred, labels=[0,1] ) 
print (cm) 
print ('Sensitivity of unifocal pvc: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of multifocal pvc: %.6f'%float(cm[1][1]/np.sum(cm[1])))

best_loss = 0.109405
Accuracy: 0.892857
[[ 5  2]
 [ 1 20]]
Sensitivity of unifocal pvc: 0.714286
Sensitivity of multifocal pvc: 0.952381


In [75]:
#Supervised Hashing Methods: Hashnet is pretrained with ground-truth samples.
#https://github.com/luyajie/triplet-deep-hash-pytorch/blob/master/hashNet.py
class HashTripletNet(nn.Module):
    def __init__(self, features=360, hashLength=36):
        super(HashTripletNet, self).__init__()
        self.fc = nn.Linear(features, hashLength)
        self.sm = nn.Sigmoid() ##nn.ReLU(inplace=True)
        self.initLinear()

    def forward(self, q, p, n):
        q = self.sm(self.fc(q))
        p = self.sm(self.fc(p))
        n = self.sm(self.fc(n))
        return q, p, n

    def initLinear(self):
        self.fc.weight.data.normal_(1.0, 0.3)
        self.fc.bias.data.fill_(0.1)
        
#generate triplet label
sub_list = list(set(np.array(tePVC['sub']).tolist()))
trQ_sf, trP_sf, trN_sf = [], [], []
for bt in sub_list:
    sub_df =  tePVC[tePVC['sub']==bt]
    y = np.array(sub_df['label']).tolist()
    if (sub_df.shape[0]>3 and len(list(set(y)))>1): #query:positive:negative
        X = sub_df.drop(['sub','label'],axis=1).reset_index(drop=True)
        X = np.array(X)
        for i in range(X.shape[0]):#query
            for j in range(X.shape[0]):#positive
                if (i!=j) and (y[i]==y[j]):
                    for k in range(X.shape[0]):#negative
                        if(y[i]!=y[k]):
                            trQ_sf.append(X[i])
                            trP_sf.append(X[j])
                            trN_sf.append(X[k])
                            break
                    break
assert (len(trQ_sf)==len(trP_sf))
assert (len(trQ_sf)==len(trN_sf))
trQ_sf = np.array(trQ_sf)
trP_sf = np.array(trP_sf)
trN_sf = np.array(trN_sf)
#training triplet model
#define model
model = HashTripletNet().cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
pdist = nn.PairwiseDistance(2)
#train model
best_net, best_loss = None, float('inf')
batchSize = 100
for epoch in range(10):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize + 1
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        Q_hash,P_hash,N_hash = model(Q_batch,P_batch,N_batch)#permute the dims of matrix
        # loss
        loss_pos = pdist(Q_hash, P_hash)
        loss_neg = pdist(Q_hash, N_hash)
        l = 18 - loss_neg + loss_pos
        loss = torch.mean(F.relu(l))#
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()

 16 / 16 : loss = 17.079418Eopch:     1 mean_loss = 17.925476
 16 / 16 : loss = 17.553492Eopch:     2 mean_loss = 17.909584
 16 / 16 : loss = 17.915691Eopch:     3 mean_loss = 17.906256
 16 / 16 : loss = 17.647793Eopch:     4 mean_loss = 17.887211
 16 / 16 : loss = 17.057114Eopch:     5 mean_loss = 17.857051
 16 / 16 : loss = 17.597784Eopch:     6 mean_loss = 17.856803
 16 / 16 : loss = 18.378519Eopch:     7 mean_loss = 17.865483
 16 / 16 : loss = 17.742271Eopch:     8 mean_loss = 17.835155
 16 / 16 : loss = 17.884775Eopch:     9 mean_loss = 17.829639
 16 / 16 : loss = 17.814701Eopch:    10 mean_loss = 17.814885
best_loss = 17.814885


In [79]:
#Testing hamming distance with true label
sub_list = list(set(np.array(tePVC['sub']).tolist()))
y_pred, y_true = [], []
pdist = nn.PairwiseDistance(2)
for bt in sub_list:
    sub_df =  tePVC[tePVC['sub']==bt]
    if (sub_df.shape[0]>1): 
        y = list(set(np.array(sub_df['label']).tolist()))
        if len(y)==1: y_true.append(0) #unifocal
        else:  y_true.append(1) #multifocal
        #predict
        X = sub_df.drop(['sub','label'],axis=1).reset_index(drop=True) 
        X = np.array(X)
        k = 1
        for i in range(X.shape[0]-1):
            Q_batch = torch.from_numpy(X[i:i+1]).type(torch.FloatTensor).cuda()
            N_batch = torch.from_numpy(X[i+1:i+2]).type(torch.FloatTensor).cuda()
            Q_hash,_,N_hash = best_net(Q_batch,Q_batch,N_batch)
            #loss_neg = torch.mean(F.relu(pdist(Q_hash, N_hash))).item()
            loss_neg = torch.mean(pdist(Q_hash, N_hash)).item()
            if loss_neg>0.2: 
                k = k+1
                break
        if k==1: y_pred.append(0) #unifocal
        else:  y_pred.append(1) #multifocal
            
print ( 'Accuracy: %.6f'%accuracy_score(y_true, y_pred))
cm = confusion_matrix(y_true, y_pred, labels=[0,1] ) 
print (cm) 
print ('Sensitivity of unifocal pvc: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of multifocal pvc: %.6f'%float(cm[1][1]/np.sum(cm[1])))

Accuracy: 0.642857
[[ 4  3]
 [ 7 14]]
Sensitivity of unifocal pvc: 0.571429
Sensitivity of multifocal pvc: 0.666667


In [3]:
#Testing Inverted index with true label
def genInvertedIndex(X, bin_len=0.01):
    # parameter: X ,numpy array (n*m)
    # bin_len, float, discretize the continuous value with bins
    # output: X_i, numpy array (w*n), w is the length of bins
    con_values = sorted(list(set(X.flatten().tolist())))
    bins = np.arange(con_values[0],con_values[-1],bin_len)
    X_i = np.zeros((len(bins)+1,X.shape[0]+1))
    for i in range(X.shape[0]):
        for val in X[i].tolist():
            X_i[np.digitize(val,bins),i]=1
    return X_i
    
sub_list = list(set(np.array(tePVC['sub']).tolist()))
y_pred, y_true = [], []
for bt in sub_list:
    sub_df =  tePVC[tePVC['sub']==bt]
    if (sub_df.shape[0]>1): 
        y = list(set(np.array(sub_df['label']).tolist()))
        if len(y)==1: y_true.append(0) #unifocal
        else:  y_true.append(1) #multifocal
        #calculate hanmming distance
        X = sub_df.drop(['sub','label'],axis=1).reset_index(drop=True) 
        X = np.array(X)
        X_i = genInvertedIndex(X)
        k = 1
        for i in range(X_i.shape[1]):
            for j in range(i+1,X_i.shape[1]):
                dist = pdist(np.vstack([X_i[:,i],X_i[:,j]]),'hamming')
                if dist>0.3: k = k+1
        if k==1: y_pred.append(0) #unifocal
        else:  y_pred.append(1) #multifocal
            
print ( 'Accuracy: %.6f'%accuracy_score(y_true, y_pred))
cm = confusion_matrix(y_true, y_pred, labels=[0,1] ) 
print (cm) 
print ('Sensitivity of unifocal pvc: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of multifocal pvc: %.6f'%float(cm[1][1]/np.sum(cm[1])))

Accuracy: 0.750000
[[ 0  7]
 [ 0 21]]
Sensitivity of unifocal pvc: 0.000000
Sensitivity of multifocal pvc: 1.000000
